In [1]:
# Importing packages
import os
import pandas as pd
import numpy as np
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.metadata.visualizers import tabulate

from qiime2.plugins import feature_classifier
from qiime2.plugins import metadata
from qiime2.plugins import taxa

from qiime2.plugins.taxa.methods import collapse
from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.methods import relative_frequency

import matplotlib.pyplot as plt
import seaborn as sns

from  statannot  import  add_stat_annotation

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set_theme(style="ticks", palette="pastel")

In [2]:
# install scikit-learn specific version to use trained classifier
%pip install --user 'scikit-learn==0.23.1'
#%pip install -U scikit-learn

  Using cached scikit_learn-0.23.1-cp38-cp38-manylinux1_x86_64.whl (6.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Parameters
experiment_name = "thayane-PM-joined"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
manifest_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/manifest-single.csv"
)
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/metadata.tsv"
class_col = "above_10"
classifier_file = (
    "/home/lauro/nupeb/rede-micro/models/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
)
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6

In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))
sheet_folder = os.path.abspath(os.path.join(experiment_folder, 'sheets'))
!mkdir -p {sheet_folder}

In [5]:
tax_level = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reps_path = os.path.join(qiime_folder, 'dada2-reps.qza')
dada2_stat_path = os.path.join(qiime_folder, 'dada2-stat.qza')

# Input - Taxonomy Classifier - SILVA 138-99
classifier_path = classifier_file
# classifier_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'models', 'silva-138-99-nb-classifier.qza'))

# Output - Excel file
excel_path = os.path.join(sheet_folder, 'abundances.xlsx')

# Output - Metataxonomy Artifact
metatax_path = os.path.join(qiime_folder, 'metatax.qza')
metatax_view_path = os.path.join(qiime_folder, 'metatax.qzv')
metatax_bar_path = os.path.join(qiime_folder, 'metatax-bar.qzv')

# Flag - Load or create files
need_tax = not (os.path.isfile(metatax_path)) or replace_files
need_view = not (os.path.isfile(metatax_view_path) or os.path.isfile(metatax_bar_path)) or replace_files

In [6]:
metadata_qa = Metadata.load(metadata_file)
metadata_df = metadata_qa.to_dataframe()
groups_values = list(metadata_qa.get_column(class_col).to_series().unique())
# tabulate(metadata_qa).visualization

In [7]:
readytowear_path = os.path.abspath(os.path.join(base_dir, '..', 'readytowear'))
dataset_path = os.path.join(readytowear_path, 'data', 'silva_138', '515f-806r')
dataset_ref_seqs_path = os.path.join(dataset_path, 'ref-seqs.qza')
dataset_ref_tax_path = os.path.join(dataset_path, 'ref-tax.qza')
dataset_ref_weight_path = os.path.join(dataset_path, 'human-stool.qza')

custom_classifier_path = os.path.join(os.path.dirname(classifier_path), 'silva_138_human-stool_classifier.qza')

In [8]:
bash_args = [dataset_ref_seqs_path, dataset_ref_tax_path, dataset_ref_weight_path, custom_classifier_path]
for a in bash_args:
    !ls -lah {a}

-rw-rw-r-- 1 lauro lauro 14M Aug 22 18:02 /home/lauro/nupeb/rede-micro/readytowear/data/silva_138/515f-806r/ref-seqs.qza
-rw-rw-r-- 1 lauro lauro 5.4M Aug 22 18:02 /home/lauro/nupeb/rede-micro/readytowear/data/silva_138/515f-806r/ref-tax.qza
-rw-rw-r-- 1 lauro lauro 2.1M Aug 22 18:02 /home/lauro/nupeb/rede-micro/readytowear/data/silva_138/515f-806r/human-stool.qza
-rw-rw-r-- 1 lauro lauro 147M Aug 30 23:58 /home/lauro/nupeb/rede-micro/models/16S_classifiers_qiime2/silva_138_human-stool_classifier.qza


In [9]:
# %%bash -s "$dataset_ref_seqs_path" "$dataset_ref_tax_path" "$dataset_ref_weight_path" "$custom_classifier_path"

# # ls -lah ${1}
# # ls -lah ${2}
# # ls -lah ${3}
# # ls -lah ${4}

# qiime feature-classifier fit-classifier-naive-bayes \
#   --i-reference-reads ${1} \
#   --i-reference-taxonomy ${2} \
#   --i-class-weight ${3} \
#   --o-classifier ${4}
#   --verbose

In [10]:
from qiime2.plugins.feature_classifier.methods import fit_classifier_naive_bayes

full_lenght = False
dataset_type = ('silva_138', 'gg_13_8')[0]
size_type = 'full_length' if full_lenght else'515f-806r'
weight_type = ('human-stool', )[0]


# Define paths
readytowear_path = os.path.join(base_dir, '..', 'readytowear')
dataset_path = os.path.join(readytowear_path, 'data', dataset_type, size_type)
dataset_ref_seqs_path = os.path.join(dataset_path, 'ref-seqs.qza')
#dataset_ref_seqs_path = "/home/lauro/nupeb/rede-micro/models/silva-138-99-seqs.qza"
dataset_ref_tax_path = os.path.join(dataset_path, 'ref-tax.qza')
#dataset_ref_tax_path = "/home/lauro/nupeb/rede-micro/models/silva-138-99-tax.qza"
dataset_ref_weight_path = os.path.join(dataset_path, f'{weight_type}.qza')

custom_classifier_path = os.path.join(os.path.dirname(classifier_path), f'{dataset_type}_{weight_type}_classifier.qza')


# Load data
dataset_ref_seqs = Artifact.load(dataset_ref_seqs_path)
dataset_ref_tax = Artifact.load(dataset_ref_tax_path)
dataset_ref_weight = Artifact.load(dataset_ref_weight_path)


# Set DataFrames
dataset_ref_tax_df = dataset_ref_tax.view(Metadata).to_dataframe()
dataset_ref_seqs_df = dataset_ref_seqs.view(Metadata).to_dataframe()
dataset_ref_weight_df = dataset_ref_weight.view(Metadata).to_dataframe()
print(f'{dataset_ref_seqs_df.shape} - {dataset_ref_tax_df.shape} - {dataset_ref_weight_df.shape}')

(313734, 1) - (313734, 1) - (1, 67689)


In [11]:
try:
    # Loading a trained probabilistic Naive Bayes model 
    classifier_naive_bayes = Artifact.load(custom_classifier_path)
except:
    # Traning probabilistic Naive Bayes model 
    classifier_naive_bayes 	= fit_classifier_naive_bayes(
        reference_reads 	= dataset_ref_seqs, # FeatureData[Sequence]
        reference_taxonomy	= dataset_ref_tax, # FeatureData[Taxonomy]
        class_weight		= dataset_ref_weight, # FeatureTable[RelativeFrequency], optional
        verbose = True,
    ).classifier
    classifier_naive_bayes.save(custom_classifier_path)

In [12]:
# Load FeatureData[Sequence] Artifact
reps = Artifact.load(dada2_reps_path)

# Load FeatureTable[Frequency] Artifact
tabs = Artifact.load(dada2_tabs_path)
asv_df = tabs.view(pd.DataFrame)

In [13]:
if not need_tax:
    
    # Load FeatureData[Taxonomy]
    metatax_qa = Artifact.load(metatax_path)
    
else:
    
    # Load TaxonomicClassifier Artifact
    classifier = Artifact.load(classifier_path)
    
    # Classify ASV features and create a new FeatureData[Taxonomy]
    metatax_qa = feature_classifier.methods.classify_sklearn(reads=reps, classifier=classifier_naive_bayes, n_jobs=threads).classification

    # Save FeatureData[Taxonomy] Artifact
    metatax_qa.save(metatax_path)

metatax_df = metatax_qa.view(pd.DataFrame)

In [14]:
metatax_naive_qa_path = os.path.join(qiime_folder, f'{dataset_type}_{weight_type}_metatax.qza')
if not os.path.exists(metatax_naive_qa_path):
    metatax_naive_qa = feature_classifier.methods.classify_sklearn(reads=reps, classifier=classifier_naive_bayes, n_jobs=threads).classification
    metatax_naive_qa.save(metatax_naive_qa_path)
else:
    metatax_naive_qa = Artifact.load(metatax_naive_qa_path)
metatax_naive_qa

{'reads_per_batch': 'auto', 'n_jobs': 6, 'pre_dispatch': '2*n_jobs', 'confidence': 0.7, 'read_orientation': 'auto', 'reads': <q2_types.feature_data._format.DNAFASTAFormat object at 0x7f767f03da00>, 'classifier': Pipeline(steps=[('feat_ext',
                 HashingVectorizer(alternate_sign=False, analyzer='char_wb',
                                   n_features=8192, ngram_range=(7, 7))),
                ['classify',
                 LowMemoryMultinomialNB(alpha=0.001,
                                        class_prior=(1.4773449157174732e-11,
                                                     1.4773449157174732e-11,
                                                     7.313934443004958e-08,
                                                     1.4773449157174732e-11,
                                                     1.4773449157174732e-11,
                                                     1.4773449157174732e-11,
                                                     1.4773449157

<artifact: FeatureData[Taxonomy] uuid: cbf75a41-6003-481e-a082-945eed71fadb>

In [15]:
metatax_naive_qv = metadata.visualizers.tabulate(metatax_naive_qa.view(Metadata))
metatax_naive_qv.visualization.save(os.path.join(qiime_folder, f'{dataset_type}_{weight_type}_metatax.qzv'))

# Barplot Visualization
# Next, we can view the taxonomic composition of our samples with interactive bar plots. 
# Generate those plots with the following command and then open the visualization.
metatax_naive_bar_qv = taxa.visualizers.barplot(tabs, metatax_naive_qa, metadata_qa)
metatax_naive_bar_qv.visualization.save(os.path.join(qiime_folder, f'{dataset_type}_{weight_type}_metatax_bar.qzv'))

{'input': Metadata
--------
1736 IDs x 2 columns
Taxon:      ColumnProperties(type='categorical')
Confidence: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'page_size': 100}
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'table': 1736 x 44 <class 'biom.table.Table'> with 2090 nonzero entries (2% dense), 'taxonomy': Feature ID
00051a78b3e1ecd27314247dff25a208    d__Bacteria; p__Firmicutes; c__Clostridia; o__...
000f8bfd8a4b2441d868821eef660ecc    d__Bacte

'/home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-joined/qiime-artifacts/silva_138_human-stool_metatax_bar.qzv'

In [16]:
metatax_df = metatax_qa.view(pd.DataFrame)
metatax_df

,Taxon,Confidence
Feature ID,,
00051a78b3e1ecd27314247dff25a208,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.9488269323665718
000f8bfd8a4b2441d868821eef660ecc,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.8661924544634567
0053504a75cbad644658d68e0d32aa7e,d__Bacteria,0.9987023990949311
00d94a1863ec0f2c06c4ad049604e60e,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.9468840020912782
010045ee9c16bba62338fd5d55530b6a,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.9998942428574114
...,...,...
ffc62661b58a6f59ad20e7ad34405896,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.9938381044094249
ffc9b5e055dca60dab5f464147875e3a,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.9971181754570829
ffca45f65366280127b61f681a7b9d7f,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.9816544470032202
